#The Effect of Humour in Political Messaging: 
An investigation combining fine-tuned neural language models and social network analysis<br>
by<br>
Janice Butler: University of Amsterdam, Master Thesis 2021

## Introduction
This notebook implements the fine-tuning of various neural language models (NLMs) based on a new corpus of annotated humorous texts.
Two classifications are made

1.   Degree of humour
2.   Comic styles

## Method 
The training script is modified from [run_glue.py](https://huggingface.co/transformers/examples.html#glue). 
The training is automatically tracked in the Weights & Biases dashboard. 

### Supervised Fine-Tuning

This script fine-tunes NMLs on corpora scraped from several sub-reddits:
* https://www.reddit.com/r/Jokes/
* https://www.reddit.com/r/satire/
* https://www.reddit.com/r/Showerthoughts/
* https://www.reddit.com/r/SurrealHumor

and from twitter:
* https://twitter.com/midnight

For non-humorous texts an equal amount of data was taken from these serious news outlets:
* https://twitter.com/AP
* https://twitter.com/BBCworld
* https://twitter.com/ITN
* https://twitter.com/ITVnews
* https://twitter.com/SkyNewsPolitics
* https://twitter.com/TheEconomist

### Annotation
The reddit data is automatically annotated into 5 grades  according to the up-votes per subreddit. All other annotation was achieved through manual categorisation, for humour degree on the https://twitter.com/midnight tweets and in all cases for type of humour. The categories being:
* Serious
* Fun
* Benevolent humour
* Wit
* Nonsense
* Irony
* Satire
* Sarcasm
* Cynicism

## Install dependencies

Pre-Trained Neural Language Models (NLMs) are taken from [the repository at Huggingface](https://huggingface.co/models). The generic Huggingface [Transformers API](https://huggingface.co/transformers/)  is used throughout for fine-tuning and the [Huggingface Pipeline API](https://huggingface.co/transformers/main_classes/pipelines.html) is taken for easy utilisation of the finished models

### NLM Training-Performance Monitoring
During fine-tuning a multitude of parameters are relayed to a data-base at https://wandb.ai/site. Additionally the fine-tuned model and all resultant meta-data for later cataloging of results and use with the model are recorded in projects defined [here](https://wandb.ai/jb-diplom) 
 

```
TODO: add screenshot loaded from GIT
```



Install the Hugging Face transformers and Weights & Biases libraries, and the dataset and training script for humour fine-tuning.

## Installation and Import of Required Packages and Libraries

The dependencies are as follows:


* Huggingface framework for loading and training models, preprocessing of data
* Optionally install transformers datasets, but not needed if own data/project data is being used
* Wandb is used for visualization of results on the project dashboard https://wandb.ai/jb-diplom/janice-demo
* sentencepiece is required for deberta models
* General purpose libraries (os, glob, pandas, numpy)
* GUI and visualization libraries (data_table, ipywidgets, plotly, tqdm, matplotlib
* For calculating accuracy of fine-tuned models and visualizing the results , sklearn.metrics is used



In [ ]:
#@markdown Do imports
!pip install transformers -qq           # huggingface framework for loading and training models, preprocessing of data
# Uncomment following line to carry out benchmark tests with hf datasets
!pip install transformers datasets -qq  # currently transformers datasets --> add own data
!pip install wandb -qq                  # for visualization of results on the project dashboard https://wandb.ai/jb-diplom/janice-demo
!pip install sentencepiece              # required for deberta
!pip install chart_studio
# this was the basis for the inital imlementation
# !wget https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py -qq

# Weights and Biases logging of training metrics and archiving of training results
import wandb

# General purpose libraries
from   google.colab import drive
import glob
import os
import pandas as pd
import numpy as np

# Visualization libraries
%load_ext google.colab.data_table
from   google.colab import data_table
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from   tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt # For multi plots

# Hugging face API for loading pre-trained models, fine-tuning and utilization
import transformers
from   transformers import AutoModelForSequenceClassification, AutoConfig, pipeline

# Stuff for calculating accuracy of fine-tuned models
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn import metrics

# Stuff for displaying metrics of fine-tuned models
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import PrecisionRecallDisplay

## API Key
The following calls registers this run at Weights and Biases github unless a session is already active.
Optionally, we can set environment variables to customize W&B logging. See [documentation](https://docs.wandb.com/library/integrations/huggingface).

### Google Drive
The project data is hosted on GDrive to enable an easy interface with [Google Colab](https://colab.research.google.com/). The training data and results are taken from and stored to directories on GDrive, which has to be mounted and requires appropriate credentials.

In [ ]:
#@markdown Connect to wandb
wandb.login()

In [ ]:
#@markdown Mount GDrive
drive.mount('/content/gdrive',True)
file_list = glob.glob("/content/gdrive/MyDrive/ColabNotebooks/Visualization/data/*")

In [ ]:
#@markdown Set some global values for consistency of output styling
plot_bgcolor='rgb(150,150,160)'
cmap='viridis'
color_palette_r = px.colors.sequential.Viridis_r
color_palette   = px.colors.sequential.Viridis

# Specify Parameters and Train Model

Here you can choose which pre-trained NL model to fine-tune. Further options are:

*   Which training-data to use
*   Which project to save run-time data to
*   The GLUE-Task to use
*   Initial learning rate
*   Number of epochs to train
*   Stepsize for logging
*   Whether to freeze layers
*   Testrun with mini dataset or not









In [ ]:
#@title Enter Parameters for Training { vertical-output: true, form-width: "50%", display-mode: "form" }

#@markdown Specify Parameters for Training
#@markdown ---

# Take viable names from https://huggingface.co/transformers/pretrained_models.html
Comment = "distilbert-midnight_roughtest" #@param {type:"string"}
Model = "distilbert-base-uncased" #@param ["bert-base-uncased", "distilbert-base-uncased", "gpt2", "distilgpt2", "gpt2-medium", "xlnet-base-cased", "roberta-base", "distilroberta-base", "t5-base", "microsoft/deberta-base", "google/electra-base-discriminator","google/electra-large-discriminator", "vinai/bertweet-base"] {allow-input: true}
GLUE_Task = "" #@param ["","cola", "mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli"] {allow-input: false}
Initial_Learn_Rate = 2e-5 #@param {type: "number"}

NrEpochs =   5#@param {type: "number"}
Do_Train = True #@param {type:"boolean"}
Do_Eval = True #@param {type:"boolean"}
Do_Predict = True #@param {type:"boolean"}
 

#@markdown ---
#@markdown Parameters for Quick Tests
#@markdown ---
do_quick_test = True #@param {type:"boolean"}
Freeze_Layers = False #@param {type:"boolean"}
max_train_samples = 700 #@param {type:"slider", min:100, max:10000, step:100}
max_val_samples   = 60 #@param {type:"slider", min:10, max:1000, step:10}
max_test_samples  = 50 #@param {type:"slider", min:10, max:1000, step:10}
#Percent_of_Trainingdata_to_use = 10 #@param {type:"slider", min: 5, max:100, step:5}

#@markdown ---
#@markdown Visualization Parameters
#@markdown ---

Do_Visualization = True #@param {type:"boolean"}
WandB_Project = "janice-roughtests" #@param ["thesis", "thesis-test-runs"] {allow-input: true}
Logging_Steps = 20 #@param {type:"slider", min:10, max:100, step:10}

#@markdown Choose Files for Training 
#@markdown ---
file_ext = ".tsv" #@param [".tsv", ".csv", ".json"] {allow-input: true}


In [ ]:
#@markdown Which files would you like to use for traiing the NLM?

dir_list = glob.glob("/content/gdrive/MyDrive/ColabNotebooks/Visualization/*/")
dir_choice = widgets.Dropdown(options=dir_list,value=dir_list[0])
# file_list = glob.glob("/content/gdrive/MyDrive/ColabNotebooks/Visualization/data/*"+ file_ext)
# file_list = glob.glob("/content/gdrive/MyDrive/ColabNotebooks/Visualization/msc_data/*"+ file_ext)
file_list = glob.glob("/content/gdrive/MyDrive/ColabNotebooks/Visualization/midnight/*"+ file_ext)

file_list.insert(0,"")
train_file = widgets.Dropdown(options=file_list,value="")
validation_file = widgets.Dropdown(options=file_list,value="")
test_file = widgets.Dropdown(options=file_list,value="")

items = [widgets.Label(value="Source Directory"),
         widgets.Label(value= "Training"),
         widgets.Label(value="Validation"),
         widgets.Label(value="Test")]

left_box = widgets.VBox([items[0], items[1], items[2], items[3]],width='10%')
right_box = widgets.VBox([dir_choice,train_file,validation_file,test_file],width='80%')
file_pickers=widgets.HBox([left_box, right_box], width='100%')
# file_pickers.overflow_x = 'auto'
right_box.overflow_x = 'auto'

def updateDoclist(b):
    train_file.options=glob.glob(dir_choice.value + "*train*" + file_ext)
    validation_file.options=glob.glob(dir_choice.value + "*dev*" + file_ext)
    test_file.options=glob.glob(dir_choice.value + "*test*" + file_ext)

dir_choice.observe(updateDoclist, names='value')
display(file_pickers)
updateDoclist(None)


In [ ]:
os.environ['TRAINING_FILE']=train_file.value
os.environ['VALID_FILE']=validation_file.value
os.environ['TEST_FILE']=test_file.value
os.environ['WANDB_PROJECT']=WandB_Project 
os.environ['WANDB_LOG_MODEL'] = 'true'  # saving the model to wandb
os.environ['GLUE_TASK_NAME']=GLUE_Task
os.environ['TRAIN_EPOCHS']=str(NrEpochs)
os.environ['MODEL']=Model
os.environ['LR']=str(Initial_Learn_Rate)
os.environ['LS']=str(Logging_Steps)
os.environ['RUNNAME']=Comment
os.environ['REPORT_TO']=""
os.environ['OUTPUT_DIR']="/content/gdrive/MyDrive/ColabNotebooks/SavedModels/"+Model
os.environ['WANDB_WATCH']="all"
os.environ['SAVE_STEPS']="5000" # big step to avoid filling disk quota
os.environ['SAVE_LIMIT']="1"    # only one backup (let's live dangeraously but save space)
os.environ['BATCH_SIZE']="12"    
os.environ['SEQ_LENGTH']="256"    


# to restart from checkpoint use following type of model path
# os.environ['MODEL']="/content/gdrive/MyDrive/ColabNotebooks/SavedModels/"+Model+"/checkpoint-12000/"
if (Do_Visualization):
  os.environ['REPORT_TO']="wandb"

# %env
#  --task_name $GLUE_TASK_NAME \
#  --jb_task_name "t5" \

if do_quick_test:
  os.environ['TRAIN_SAMPLES']=(str(max_train_samples) if do_quick_test else "")
  os.environ['VAL_SAMPLES']=  (str(max_val_samples)   if do_quick_test else "")
  os.environ['TEST_SAMPLES']= (str(max_test_samples)  if do_quick_test else "")
  
  !python '/content/gdrive/MyDrive/ColabNotebooks/Visualization/run_glue2.py' \
    --model_name_or_path $MODEL \
    --max_val_samples $VAL_SAMPLES \
    --max_test_samples $TEST_SAMPLES \
    --max_train_samples $TRAIN_SAMPLES \
    --tokenizer_name $MODEL \
    --do_train \
    --do_eval \
    --do_predict \
    --max_seq_length $SEQ_LENGTH \
    --per_device_train_batch_size $BATCH_SIZE \
    --per_device_eval_batch_size=64 \
    --learning_rate $LR \
    --num_train_epochs $TRAIN_EPOCHS \
    --output_dir $OUTPUT_DIR \
    --overwrite_output_dir \
    --logging_steps $LS \
    --run_name $RUNNAME \
    --report_to $REPORT_TO \
    --train_file $TRAINING_FILE \
    --validation_file $VALID_FILE \
    --test_file $TEST_FILE \
    --save_steps $SAVE_STEPS \
    --save_total_limit $SAVE_LIMIT \
    --fp16 \
    --adafactor --lr_scheduler_type cosine --warmup_ratio 0.1 \
    --skip_memory_metrics
else:
  !python '/content/gdrive/MyDrive/ColabNotebooks/Visualization/run_glue2.py' \
    --model_name_or_path $MODEL \
    --tokenizer_name $MODEL \
    --do_train \
    --do_eval \
    --do_predict \
    --max_seq_length $SEQ_LENGTH \
    --per_device_train_batch_size $BATCH_SIZE \
    --per_device_eval_batch_size=64 \
    --learning_rate $LR \
    --num_train_epochs $TRAIN_EPOCHS \
    --output_dir $OUTPUT_DIR \
    --overwrite_output_dir \
    --logging_steps $LS \
    --run_name $RUNNAME \
    --report_to $REPORT_TO \
    --train_file $TRAINING_FILE \
    --validation_file $VALID_FILE \
    --test_file $TEST_FILE \
    --save_steps $SAVE_STEPS \
    --save_total_limit $SAVE_LIMIT \
    --fp16 \
    --adafactor --lr_scheduler_type cosine --warmup_ratio 0.1 \
    --skip_memory_metrics


# Testing of fine-tuned models
Retrieve model from W&B repository

In [ ]:
# run = wandb.init()
run= wandb.init(project=WandB_Project, entity='jb-diplom')
# Take this from just finished run or from one of your 
# favorite fine-tuned models at https://wandb.ai/jb-diplom/janice-full/artifacts

#@markdown Enter Model Name to be retreived from https://wandb.ai
model_id = '1ap79mgp'  #@param {type: "string"}

model_root= 'jb-diplom/' + WandB_Project + '/model-'
model_path= model_root + model_id + ':v0'
print("Retreiving artefact:", model_path)
artifact = run.use_artifact(model_path, type='model')
# artifact = run.use_artifact('jb-diplom/janice-full/model-219xio3e:v0', type='model')
artifact_dir = artifact.download()
print("Model saved locally to:", artifact_dir)

In [ ]:
print("Model saved locally to:", artifact_dir)

In [ ]:
# Load fine-tuned model
# model_path="/content/gdrive/MyDrive/ColabNotebooks/SavedModels/"+Model
# model_path="/content/gdrive/MyDrive/ColabNotebooks/SavedModels/distilbert-base-uncased"

# model_path="/content/artifacts/model-39rcocd0:v0" # A good bert-large with 0-3 labels

model_path='/content/artifacts/model-' + model_id + ':v0' # The model just downloaded from wandb.io

# This text classification pipeline can currently be loaded from pipeline() using the following task 
# identifier: "sentiment-analysis"

#@markdown Enter a test string to check whether the downloaded model is working
test_string = 'i shot a bullet into the air and it hit my hand. on one hand i\u2019m really happy that it didn\u2019t hit my head and kill me, but on the other hand i have a big gaping hole now.'  #@param {type: "string"}

humour_classif = pipeline('sentiment-analysis',model_path)
humour_classif(test_string)

In [ ]:
#@markdown Load file (specified for testing above) for manual testing
# Take test.tsv and compare expected with actual results
test_df = pd.read_csv(test_file.value, delimiter='\t', header=None, 
                        lineterminator='\n',encoding='utf-8')
cols=['Text','Humour Level']
test_df.columns=cols
# test_df.describe()
data_table.DataTable(test_df, include_index=False, num_rows_per_page=10)

In [ ]:
# Do a little test against a chosen test-dataset
#@markdown #### **Attention:** This Operation may take many hours, depending on the quantity of test data!
#@markdown ----
#@markdown ###How many records would you like to test?
#@markdown An entry of -1 implies processing of **ALL** records

sample_nr =   100#@param {type: "number"}

# tweet_df.iloc[1:5, 1:1]
content=test_df.iloc[:,0]
labels=test_df.iloc[:,1]
humour=[]
y_true=[]
y_score=[]

# establish humour content of tweet from fine-tuned model
hit=0
miss=0
out_by_one=0

for i, tweet in tqdm(enumerate(content.head(sample_nr)),total=(len(content) if sample_nr==-1 else sample_nr)):
  # clip to max_seq_length, extract the number from the label of the result 
  cls_val=humour_classif('{:1.512}'.format(tweet))
  val=cls_val[0]['label'][6:7:1]
  try:
    if (int(val) == int(labels[i][0:1:1])):
      y_true.append(1)
      hit +=1
    else:
      miss +=1
      y_true.append(0)
      if abs(int(val) - int(labels[i][0:1:1])) <3:
        out_by_one += 1
    humour.append(val)
    y_score.append(cls_val[0]['score'])
  except:
    continue

print ("Hits:", hit,"\nMisses:",miss,"\nOut by one:", out_by_one, "\n%-age of hits:", (hit*100)/(hit+miss))
# '{:1.35}'.format('12345678901234567890') use to truncate string


In [ ]:
#@title Analyse Results of Training

#@markdown To assess the statistica success of the fine-tuning, 3 analyses are conducted using the test data-set:
#@markdown * Confusion Matrix<br>
#@markdown  **TODO** add descriptions 
#@markdown * ROC<br>
#@markdown  **TODO** add descriptions 
#@markdown * Precision Recall<br>
#@markdown  **TODO** add descriptions
#@markdown 

names=['serious','fun','benevolent','wit','nonsense','irony','satire','sarcasm','cynicism']
clean_h=[]
for val in humour:
  clean_h.append(val.replace('\r',""))
clean_p=[]
pred=test_df['Humour Level'].tolist()[1:len(clean_h)+1:1]
for val in pred:
  clean_p.append(val.replace(' ',"").replace('\r',""))

cm=confusion_matrix(clean_p,clean_h, labels=['0','1','2','3','4','5','6','7','8'], normalize= 'pred')
cm_display = ConfusionMatrixDisplay(cm, display_labels=names)
# fig, ax = plt.subplots(figsize=(10,10))

fpr, tpr,  _ = roc_curve(y_true, y_score, pos_label=1)
roc_auc = metrics.auc(fpr, tpr)
# fig2, ax2 = plt.subplots(figsize=(10,10))
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name="")

avg_precision = average_precision_score(y_true, y_score)
prec, recall, _ = precision_recall_curve(y_true, y_score, pos_label=1)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall,average_precision=avg_precision,estimator_name="" )

# Try a multi plot :-)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))

cm_display.plot(ax=ax1,cmap='magma')
roc_display.plot(ax=ax2)
pr_display.plot(ax=ax3)
plt.show()

# TODO : extend this to multi-class precision recall. See example here
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py


In [ ]:
#@title Save Results to GDrive
#@markdown The results are added as column ***Humour*** to the test (input) data and are saved with a suffix (the model name) as `.tsv` file in the same directory

content=tweet_df.iloc[:,0]
humour=[]

for tweet in tqdm(content.head(sample_nr)):
  humour.append(humour_classif(tweet)[0]['label'])

hlen=len(humour)
for i in range (hlen, len(tweet_df)):
  humour.append("not evaluated")

# add humour column to dataframe
tweet_df['Humour']=humour

# save results
tweet_file_out=basedir + file_name +'_' + model_id + ext
print ("Saving to:", tweet_file_out)

# tweet_df.to_csv(tweet_file_out, sep='\t', index=False, lineterminator='\n',encoding='utf-16')
tweet_df.to_csv(tweet_file_out, sep='\t', index=False,encoding='utf-16')

data_table.DataTable(tweet_df.head(min(sample_nr,20000)), include_index=False, num_rows_per_page=10)

In [ ]:
# Read in tweets, do humour evaluation and write results back to df and save new csv
#@title Optionally Enter Alternative File Name for Conducting Test{ vertical-output: true, form-width: "50%", display-mode: "form" }

#@markdown #### Specify Source for Humour Tests (skip this if you wish to continue with the results obtained from the previous step)
#@markdown ---

basedir="/content/gdrive/MyDrive/ColabNotebooks/Visualization/msc_data/"#@param {type:"string"}
file_name='comedian-tweets250_2232a9v4'#@param {type:"string"}
ext=".csv" #@param {type:"string"}
tweet_file=basedir + file_name + ext
dtyps={'tweetId':str, 'content':str, 'username':str,'followers':int, 'conversationId':str, 
         'replyCount':int, 'retweetCount':int, 'likeCount':int, 'quoteCount':int}

# tweet_df = pd.read_csv(tweet_file, delimiter='\t', header=None, dtype=dtyps,lineterminator='\n',encoding='utf-16')
# tweet_df = pd.read_csv(tweet_file, delimiter='\t', header=None, dtype=dtyps,encoding='utf-16')
# cols=['tweetId', 'content', 'username','followers', 'conversationId', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount']

tweet_df = pd.read_csv(tweet_file, delimiter='\t',encoding='utf-16')
# tweet_df.pop(tweet_df.columns[0])
# tweet_df.pop(tweet_df.columns[3])

tweet_df.head()

In [ ]:
use_all_samples = True #@param {type:"boolean"}
sample_nr = 2660 #@param {type:"slider", min:10, max:5000, step:10}
output_max= sample_nr

if use_all_samples:
  sample_nr = -1
  output_max = len(tweet_df)



In [ ]:
#@title Display distribution of data for each type of humour
vc=tweet_df['Humour'].value_counts()
vc

In [ ]:
#@markdown Display resulting data
tweet_df = tweet_df.dropna()
subdf=tweet_df.query("Humour!='not evaluated' & likeCount > 5000")
data_table.DataTable(subdf.head(20000), include_index=False, num_rows_per_page=10)

In [ ]:
# General purpose libraries
from   google.colab import drive
import glob
import os
import pandas as pd
import numpy as np

# Visualization libraries
%load_ext google.colab.data_table
from   google.colab import data_table
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from   tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt # For multi plots

In [ ]:
#@title Choose Sample of Data for Visualization

# limit of absolute_max=20000 due to colab DataTable

import warnings
absolute_max=20000

names=['serious','fun','benevolent','wit','nonsense','irony','satire','sarcasm','cynicism']
as_many_as_possible = False #@param {type:"boolean"}
sample = 5000 #@param {type:"slider", min:100, max:20000, step:100}
op_max= sample

if as_many_as_possible:
  op_max = 20000

likes_sorted=np.sort(tweet_df['likeCount'])
num_tweets=len(tweet_df)
divisor = min (absolute_max, op_max)

last_like=0
if absolute_max < num_tweets:
  chunk_lst=np.array_split(likes_sorted,int(num_tweets/divisor))
  last_like=int(chunk_lst[-1][0])

tweet_df = tweet_df.dropna()
subdf=tweet_df.loc[(tweet_df['Humour'] !='not evaluated') & (tweet_df['likeCount'] > last_like)]

# add line breaks to make tooltips readable
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    subdf.content = subdf.content.str.wrap(80)
    subdf.content = subdf.content.apply(lambda x: x.replace('\n', '<br>'))

fig1 = px.scatter(subdf, x="retweetCount", y="username", size="likeCount",
                 color="Humour", hover_name="content", facet_col="Humour", size_max=100, log_x=True, 
                 category_orders = {'Humour': ['LABEL_0','LABEL_1','LABEL_2','LABEL_3','LABEL_4','LABEL_5','LABEL_6','LABEL_7','LABEL_8']},
                 color_discrete_sequence=color_palette, height=3000,opacity=0.6, facet_col_wrap=3)
fig1.update_layout(
    plot_bgcolor=plot_bgcolor,
    title="Tweet Distribution per Humour Type amongst leading Twitterers",
    xaxis_title="Retweets (log-scale)",
    yaxis_title="Twitter Handles",
  )
fig1.show()

In [ ]:
#@title Calculate averages 
#@markdown * For each type of Tweet propagation (reply, retweet, like, quote) 
#@markdown * And each humour type

table = pd.pivot_table(tweet_df, values=['replyCount', 'retweetCount', 'likeCount', 'quoteCount'], index=['Humour'],
                    aggfunc={'replyCount': np.mean,
                             'retweetCount': np.mean,
                             'likeCount': np.mean,
                             'quoteCount': np.mean
                             })
table=table.loc[(table.index != "not evaluated")]

# need to rotate 'replyCount', 'retweetCount', 'likeCount', 'quoteCount' into one column and put the averages in a new column
dic ={"humour":[], "propagation type":[],"mean value":[]}
for htype in table.iterrows():
  for i in range (0,4) : dic["humour"].append(htype[0])
  dic["propagation type"].append('likes')
  dic["mean value"].append(int(htype[1]['likeCount']))
  dic["propagation type"].append('quotes')
  dic["mean value"].append(int(htype[1]['quoteCount']))
  dic["propagation type"].append('replies')
  dic["mean value"].append(int(htype[1]['replyCount']))
  dic["propagation type"].append('retweets')
  dic["mean value"].append(int(htype[1]['retweetCount']))

df_avg=pd.DataFrame(dic)

df_avg.head(10)

In [ ]:
#@title Visualize average numbers of Tweet propagations per humour type
from plotly.graph_objs import *

# labels={'Humour': ['LABEL_0','LABEL_1','LABEL_2','LABEL_3','LABEL_4','LABEL_5','LABEL_6','LABEL_7','LABEL_8'],
#                                       'propagation type': ['quotes','replies','retweets','likes']}
labels={'Humour': "",'propagation type': ''}
fig = px.histogram(df_avg, 
                   x='propagation type', 
                   y='mean value', 
                   facet_col="humour",
                   histfunc ='avg',
                   color_discrete_sequence=color_palette_r,
                   color='propagation type',
                  #  barmode='group',
                   category_orders = {'Humour': ['LABEL_0','LABEL_1','LABEL_2','LABEL_3','LABEL_4','LABEL_5','LABEL_6','LABEL_7','LABEL_8'],
                                      'propagation type': ['likes','retweets','replies','quotes']},
                   labels=labels
                   )
fig.update_xaxes(type='category')

fig.update_layout(
    plot_bgcolor=plot_bgcolor,
    title="Average Numbers of Tweet Propagations per Humour Type",
    xaxis_title="",
    yaxis_title="Average Numbers of Tweet Propagations",
    # legend_title="Legend Title"
  )
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()

In [ ]:
#@markdown ##Visualize Distribution of Tweets per Twitter-Handle
#@markdown Select/deselect humour types in the legend to analyse deeper

subdf2=tweet_df.query("Humour!='not evaluated'")
s = subdf2.groupby("username")["followers"].sum().rank(ascending=True)
# s = subdf2.groupby("username").size().reset_index().groupby(['replyCount', 'retweetCount', 'likeCount', 'quoteCount']).sum().rank(ascending=True)


fig = px.histogram(subdf2, 
                   x='username',
                   color="Humour",
                   color_discrete_sequence=color_palette_r,
                   category_orders = {'Humour': ['LABEL_0','LABEL_1','LABEL_2','LABEL_3','LABEL_4','LABEL_5','LABEL_6','LABEL_7','LABEL_8'],
                                      'propagation type': ['likes','retweets','replies','quotes'],
                                      "username":s[s < 100000].sort_values().index.to_list()},
                   orientation='v', height=800
                      )
fig.update_layout(
    plot_bgcolor=plot_bgcolor,
    title="Numbers of Tweets amongst leading Twitterers",
    )
fig.show()

# Neuer Abschnitt

# Utility Code

In [ ]:
# Invoke to show what gpu is in use
gpu = !nvidia-smi
gpu = '\n'.join(gpu)
print(gpu)

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    

In [ ]:
%env TRAINING_FILE
from datasets import load_dataset, load_metric
datasets = load_dataset( "csv", delimiter='\t', data_files=[train_file.value, test_file.value,  validation_file.value])

len(datasets)
datasets["train"].column_names 
datasets["train"]
datasets[0]
show_random_elements(datasets,1)
train_file.value
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv(train_file.value, delimiter='\t', header=None, names=['sentence', 'label'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

In [ ]:
%env
!python '/content/gdrive/MyDrive/ColabNotebooks/Visualization/run_glue2.py' --help
train_file.value

## Visualization of results in dashboard
Analyze results (as they happen) on the project dashboard https://wandb.ai/jb-diplom/janice-demo

### To retrieve models and their metadata from wandb

1.   Go to the artifacts area of wandb (e.g. `https://wandb.ai/jb-diplom/janice-full/artifacts`)
2.   Select the API Tag, which gives the precise code (below) for downloading the model that you need.
3.   Check in the artifacts folder of Colab for the sub-folder (e.g. `model-12ai5jvy:0`) with the model.<br> Right-click it and take a copy of the path
4.   Use the path in the huggingface pipeline constructor e.g.<br>
`model_path="/content/artifacts/model-15ai5jvy:v0"`
`humour_classiffier = pipeline('sentiment-analysis',model_path)`


In [ ]:
!pip install jupyter-dash

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')